In [59]:
import os
import glob
import cv2
import numpy as np
from pathlib import Path
import sys
from collections import deque



def setup_project_path():
    current = Path.cwd()
    while not (current / 'craft').exists():
        current = current.parent
    return current
project_root = setup_project_path()
sys.path.insert(0, str(project_root))

In [61]:
def imwrite_unicode(path, img):
    ext = os.path.splitext(path)[1]
    ok, buf = cv2.imencode(ext, img)
    if not ok:
        return False
    with open(path, "wb") as f:
        f.write(buf.tobytes())
    return True

def imread_unicode(path, flags=cv2.IMREAD_COLOR):
    try:
        with open(path, "rb") as f:
            data = f.read()
        img_array = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(img_array, flags)
        return img
    except Exception as e:
        print("[imread_unicode ERROR]", e)
        return None


In [63]:
def load_centers_txt(center_path: str | Path) -> np.ndarray:
    pts = []
    with open(center_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            x_str, y_str = line.split(",")
            pts.append([float(x_str), float(y_str)])
    return np.array(pts, dtype=np.float32)


In [65]:
def draw_centers(img_bgr: np.ndarray, centers_xy: np.ndarray, radius: int = 7):
    out = img_bgr.copy()
    H, W = out.shape[:2]
    for idx, (x, y) in enumerate(centers_xy, start=1):
        xi, yi = int(round(x)), int(round(y))
        if 0 <= xi < W and 0 <= yi < H:
            cv2.circle(out, (xi, yi), radius, (0, 0, 255), -1)
            cv2.putText(out, str(idx), (xi + 8, yi - 8),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
    return out



In [67]:
def voronoi_label_map(H, W, centers, connectivity=4):
    if connectivity == 4:
        neigh = [(-1,0),(1,0),(0,-1),(0,1)]
    else:
        neigh = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]

    label = np.zeros((H, W), np.int32)
    dist  = np.full((H, W), 1e9, np.int32)
    q = deque()

    for i, (x, y) in enumerate(centers, start=1):
        sx, sy = int(round(x)), int(round(y))
        sx = int(np.clip(sx, 0, W - 1))
        sy = int(np.clip(sy, 0, H - 1))
        label[sy, sx] = i
        dist[sy, sx] = 0
        q.append((sx, sy))

    while q:
        x, y = q.popleft()
        for dx, dy in neigh:
            nx, ny = x + dx, y + dy
            if nx < 0 or ny < 0 or nx >= W or ny >= H:
                continue
            if dist[ny, nx] > dist[y, x] + 1:
                dist[ny, nx] = dist[y, x] + 1
                label[ny, nx] = label[y, x]
                q.append((nx, ny))

    return label


In [69]:
def ink_mask_auto(img_bgr: np.ndarray) -> np.ndarray:
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    _, m_bin = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, m_inv = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    ink_bin = (m_bin > 0).astype(np.uint8)  
    ink_inv = (m_inv > 0).astype(np.uint8)  

    return ink_bin if ink_bin.sum() <= ink_inv.sum() else ink_inv


In [71]:
def assign_split_ink_components(
    label_map: np.ndarray,
    img_bgr: np.ndarray,
    centers_xy: np.ndarray,
    *,
    connectivity: int = 8,
    min_comp_area: int = 8,
    w_area: float = 1.0,
    w_dist: float = 2.0,
    majority_ratio_thresh: float = 0.8,
) -> np.ndarray:
    refined = label_map.copy()
    centers_xy = np.asarray(centers_xy, dtype=np.float32)

    ink01 = ink_mask_auto(img_bgr).astype(np.uint8)
    conn = 4 if connectivity == 4 else 8

    num, cc = cv2.connectedComponents(ink01, connectivity=conn)
    if num <= 1:
        return refined

    for cid in range(1, num):
        comp_mask = (cc == cid)
        area = int(comp_mask.sum())
        if area < min_comp_area:
            continue

        labs = refined[comp_mask]
        labs = labs[labs != 0]
        if labs.size == 0:
            continue

        uniq, cnts = np.unique(labs, return_counts=True)
        if uniq.size <= 1:
            continue  # not split

        total = float(cnts.sum())
        major_idx = int(np.argmax(cnts))
        major_lab = int(uniq[major_idx])
        major_ratio = float(cnts[major_idx]) / total

        # majority guard
        if majority_ratio_thresh is not None and major_ratio >= float(majority_ratio_thresh):
            refined[comp_mask] = major_lab
            continue

        ys, xs = np.where(comp_mask)
        mx, my = float(xs.mean()), float(ys.mean())

        best_lab = None
        best_score = None
        for lab, lab_cnt in zip(uniq.tolist(), cnts.tolist()):
            cx, cy = float(centers_xy[lab - 1][0]), float(centers_xy[lab - 1][1])
            d2 = (mx - cx) ** 2 + (my - cy) ** 2
            score = (w_area * float(lab_cnt)) - (w_dist * float(d2))
            if best_score is None or score > best_score:
                best_score = score
                best_lab = int(lab)

        refined[comp_mask] = best_lab if best_lab is not None else major_lab

    return refined


In [73]:
def assign_global_ink_cc_to_majority_label(
    label_map: np.ndarray,
    img_bgr: np.ndarray,
    *,
    connectivity: int = 8,
    min_comp_area: int = 8,
    max_intrusion_pixels: int | None = 30,
    max_intrusion_ratio: float | None = None,
) -> np.ndarray:
    refined = label_map.copy()
    ink01 = ink_mask_auto(img_bgr).astype(np.uint8)
    conn = 4 if connectivity == 4 else 8

    num, cc = cv2.connectedComponents(ink01, connectivity=conn)
    if num <= 1:
        return refined

    for cid in range(1, num):
        comp = (cc == cid)
        area = int(comp.sum())
        if area < min_comp_area:
            continue

        labs = refined[comp]
        labs = labs[labs != 0]
        if labs.size == 0:
            continue

        uniq, cnts = np.unique(labs, return_counts=True)
        if uniq.size <= 1:
            continue  # already owned

        total = int(cnts.sum())
        major_idx = int(np.argmax(cnts))
        major_lab = int(uniq[major_idx])
        major_cnt = int(cnts[major_idx])
        intrusion = total - major_cnt

        if max_intrusion_pixels is not None and intrusion > int(max_intrusion_pixels):
            continue
        if max_intrusion_ratio is not None and (intrusion / float(total)) > float(max_intrusion_ratio):
            continue

        refined[comp] = major_lab

    return refined


In [75]:
def label_map_to_color(label_map):
    out = np.zeros((*label_map.shape, 3), np.uint8)
    rng = np.random.RandomState(1234)
    for l in np.unique(label_map):
        if l == 0:
            continue
        out[label_map == l] = rng.randint(50, 255, 3)
    return out


def overlay(img, col, a=0.7):
    return cv2.addWeighted(img, 1 - a, col, a, 0)


In [77]:
def run_character_segmentation_batch(
    *,
    in_dir: Path,
    center_dir: Path,

    map_out_dir: Path | None = None,        
    chars_out_dir: Path | None = None,      

    img_exts=(".png", ".jpg", ".jpeg", ".bmp", ".webp"),

    voronoi_connectivity: int = 4,
    cc_connectivity: int = 8,
    min_comp_area: int = 8,
    w_area: float = 1.0,
    w_dist: float = 2.0,
    majority_ratio_thresh: float = 0.8,

    global_cc_min_area: int = 8,
    max_intrusion_pixels: int | None = 30,
    max_intrusion_ratio: float | None = None,

    region_pad: int = 4,
    region_min_pixels: int = 50,
    region_save_fullsize: bool = False,
):
    if map_out_dir is None and chars_out_dir is None:
        raise ValueError("At least one of map_out_dir or chars_out_dir must be provided.")

    if map_out_dir is not None:
        map_out_dir.mkdir(parents=True, exist_ok=True)

    if chars_out_dir is not None:
        chars_out_dir.mkdir(parents=True, exist_ok=True)

    img_paths = sorted([p for p in in_dir.iterdir() if p.suffix.lower() in img_exts])
    print(f"[INFO] Found {len(img_paths)} images in {in_dir}")
    print(f"[INFO] map_out_dir  = {map_out_dir}")
    print(f"[INFO] chars_out_dir= {chars_out_dir}")

    for img_path in img_paths:
        stem = img_path.stem
        center_path = center_dir / f"res_{stem}_center.txt"

        if not center_path.exists():
            print(f"[SKIP] center not found: {center_path.name}")
            continue

        img = imread_unicode(img_path)
        if img is None:
            print(f"[SKIP] failed to read: {img_path.name}")
            continue

        centers = load_centers_txt(center_path)
        if centers.size == 0:
            print(f"[SKIP] empty centers: {center_path.name}")
            continue

        H, W = img.shape[:2]

        label_voro = voronoi_label_map(H, W, centers, connectivity=voronoi_connectivity)

        label_owner = assign_split_ink_components(
            label_voro, img, centers,
            connectivity=cc_connectivity,
            min_comp_area=min_comp_area,
            w_area=w_area,
            w_dist=w_dist,
            majority_ratio_thresh=majority_ratio_thresh,
        )

        label_global = assign_global_ink_cc_to_majority_label(
            label_owner, img,
            connectivity=cc_connectivity,
            min_comp_area=global_cc_min_area,
            max_intrusion_pixels=max_intrusion_pixels,
            max_intrusion_ratio=max_intrusion_ratio,
        )

        label_final = assign_split_ink_components(
            label_global, img, centers,
            connectivity=cc_connectivity,
            min_comp_area=min_comp_area,
            w_area=w_area,
            w_dist=w_dist,
            majority_ratio_thresh=majority_ratio_thresh,
        )

        if map_out_dir is not None:
            vis_final = draw_centers(
                overlay(img, label_map_to_color(label_final)),
                centers
            )
            map_path = map_out_dir / f"char_{stem}.png"
            ok = imwrite_unicode(map_path, vis_final)
            if not ok:
                print(f"[FAIL][MAP] write error: {map_path}")

        if chars_out_dir is not None:
            save_regions_as_images(
                img_bgr=img,
                label_map=label_final,
                out_dir=chars_out_dir / stem,
                filename_stem=stem,
                pad=region_pad,
                min_pixels=region_min_pixels,
                save_fullsize=region_save_fullsize,
            )

        print(f"[OK] processed: {img_path.name}")


In [79]:
def save_regions_as_images(
    *,
    img_bgr: np.ndarray,
    label_map: np.ndarray,
    out_dir: Path,
    filename_stem: str,
    pad: int = 4,
    min_pixels: int = 50,
    save_fullsize: bool = False,
):
    out_dir.mkdir(parents=True, exist_ok=True)

    H, W = label_map.shape[:2]
    labels = [int(l) for l in np.unique(label_map) if l != 0]

    for lab in labels:
        mask = (label_map == lab)
        pix_count = int(mask.sum())
        if pix_count < min_pixels:
            continue

        if save_fullsize:
            out = np.zeros_like(img_bgr)
            out[mask] = img_bgr[mask]
        else:
            ys, xs = np.where(mask)
            y0, y1 = int(ys.min()), int(ys.max())
            x0, x1 = int(xs.min()), int(xs.max())

            y0 = max(0, y0 - pad); y1 = min(H - 1, y1 + pad)
            x0 = max(0, x0 - pad); x1 = min(W - 1, x1 + pad)

            roi = img_bgr[y0:y1+1, x0:x1+1]
            roi_mask = mask[y0:y1+1, x0:x1+1]

            out = np.zeros_like(roi)
            out[roi_mask] = roi[roi_mask]

        out_path = out_dir / f"{filename_stem}_L{lab:03d}.png"
        imwrite_unicode(out_path, out)


In [81]:
run_character_segmentation_batch(
    in_dir=project_root / "images" / "images_normalized2",
    center_dir=project_root / "results" / "center",

    map_out_dir=project_root / "visualize" / "visualize_2",
    chars_out_dir=project_root / "characters",

    voronoi_connectivity=4,
    cc_connectivity=8,
    min_comp_area=8,
    w_area=1.0,
    w_dist=2.0,
    majority_ratio_thresh=0.8,

    global_cc_min_area=8,
    max_intrusion_pixels=30,
    max_intrusion_ratio=None,

    region_pad=4,
    region_min_pixels=80,
    region_save_fullsize=False,
)


[INFO] Found 8 images in D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\images\images_normalized2
[INFO] map_out_dir  = D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\visualize\visualize_2
[INFO] chars_out_dir= D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\characters
[OK] processed: test1.png
[OK] processed: test2.png
[OK] processed: test3.png
[OK] processed: test4.png
[OK] processed: test5.png
[OK] processed: test6.png
[OK] processed: test7.png
[OK] processed: test8.png
